In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
from pycocotools.coco import COCO
import numpy as np
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (10.0, 8.0)
from scipy.misc import imread
import matplotlib.patches as patches
from torch.utils.data import DataLoader
import cv2
import torch
from torch.autograd import  Variable
from faster_rcnn.utils.images import imshow
from faster_rcnn.fastrcnn.bbox_transform import bbox_transform
import cPickle

In [3]:
with open('sorted_index', 'rb') as fp:
    sorted_index = cPickle.load(fp)

### Đọc dữ liệu từ MS COCO dataset


In [4]:
import os
import torchvision.transforms as transforms
from faster_rcnn.utils.datasets.mscoco.dataset import CocoData
from faster_rcnn.utils.datasets.data_generator import CocoGenerator
from faster_rcnn.utils.datasets.data_generator import Enqueuer

dataDir = './data/mscoco'
dataType = 'train2014'
annFile='%s/annotations/instances_%s.json'%(dataDir,dataType)
pre_proposal_folder = './data/mscoco/coco_proposals/MCG/'

images_dir = os.path.join(dataDir,'images', dataType)
cap = CocoData(root = images_dir, annFile = annFile)

data_gen = CocoGenerator(data=cap, sorted_index=sorted_index, batch_size=3)
queue = Enqueuer(generator=data_gen)
queue.start(max_queue_size=10, workers=2)
t = queue.get()


loading annotations into memory...
Done (t=10.17s)
creating index...
index created!


## RPN

#### Mục đích 

- Do Fast RCNN vẫn phụ thuộc vào Selective search để tính toán các proposal region dẫn đến kết quả tính toán rất chậm
- RPN dùng để tính toán proposal region mà không cần phụ thuộc vào các thuật toán khác

#### Cách hoạt động

- Sử dụng một cửa sổ trượt trên feature map
- tạo một network nhỏ để tính
    - Phân loạt một anchor có chứa hay không chưa object (Anchor good / bad)
    - Tính toán các trị số của proposal regions (box regression)
    
- Vị trí của cửa sổ trượt trên feature map, cho biết thông tin về vị trí của anchor trên ảnh gốc

#### Input

- Hình ảnh: là tensor có kích thước

    $[batchsize, dim, im\_height, im\_width]$
    
- bounding box:
    
    $(x_1, y_1, x_2, y_2)$

    - $x_1, y_1$ : tọa độ x,y của điểm trái dưới (lower-left)
    - $x_2, y_2$ : tọa độ x,y của điểm phải trên (top - right)

#### Output

- RPN classification (anchor good / bad)
- RPN regression (anchor -> proposal)

Cùng xem định dạng của một input

In [5]:
blobs = t.next()

In [6]:
max_height = np.max([blob['tensor'].shape[2] for blob in blobs])
max_width = np.max([blob['tensor'].shape[3] for blob in blobs])

In [7]:
batch_tensor = torch.Tensor(3, 3, max_height, max_width).fill_(0).type_as(blob['tensor'])

In [8]:
for i, blob in enumerate(blobs):
    shape = blob['tensor'].shape
    batch_tensor[i,:, :shape[2], :shape[3] ]= blob['tensor']

In [9]:
total_boxes = 0
batch_boxes = np.empty((0, 4))
batch_boxes_index = np.empty((0,), dtype=np.int)

for i, blob in enumerate(blobs):
    total_boxes = blob['boxes'].shape[0]
    batch_boxes = np.vstack((batch_boxes, blob['boxes']))
    a =  np.zeros((total_boxes , ), dtype=np.int)
    a.fill(i)
    batch_boxes_index = np.concatenate((batch_boxes_index, a), axis=0)
    
print batch_boxes_index
print batch_boxes

[0 0 0 0 0 0 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
[[ 453.125       793.75        590.625       868.75      ]
 [   0.          315.625       496.875       959.375     ]
 [ 171.875       159.375       587.5         343.75      ]
 [ 400.         1421.875       596.875      1612.5       ]
 [ 312.5         634.375       515.625       709.375     ]
 [   0.          934.375        18.75       1018.75      ]
 [ 220.10050251  337.68844221  301.50753769  780.90452261]
 [  24.12060302    0.          569.84924623 1878.3919598 ]
 [  18.29268293  706.09756098  303.65853659  892.68292683]
 [ 351.2195122  1628.04878049  482.92682927 1690.24390244]
 [  10.97560976  918.29268293  585.36585366 1342.68292683]
 [   0.         1364.63414634  585.36585366 1803.65853659]
 [  18.29268293   14.63414634  563.41463415  435.36585366]
 [ 365.85365854  951.2195122   380.48780488 1031.70731707]
 [ 413.41463415 1408.53658537  435.36585366 1481.70731707]
 [ 384.14634146  951.2195122   391.46341463 1031.70731707]
 [ 4

INFO:faster_rcnn.utils.datasets.mscoco.dataset:need more than 0 values to unpack


Thử hiển thị ảnh cùng các bounding boxes

In [ ]:
blobs[0]['boxes']

In [ ]:
imshow(batch_tensor[0], blobs[0]['boxes'])

In [ ]:
imshow(batch_tensor[1], blobs[1]['boxes'])

In [ ]:
imshow(batch_tensor[2], blobs[2]['boxes'])

In [ ]:
from faster_rcnn.faster_rcnn import  RPN
rpn_network = RPN()
rpn_network.cuda()

### Tính toán feed-forward


Chúng ta sử dụng một ảnh có kích thước đầu vào là  `(width , height) = (600, 800)`

Input:
    - im_data : 
        kích thước : (batch_size, dim, witdh, height)
    - ground_boxes: 
        kích thước (n, 4)
        

#### Tính `cnn features`  

Sử dụng mạng CNN đã được train trên Imagenet, cụ thể là VGG16 để tính toán `cnn features`

In [ ]:
# convert tensor to variable
x = Variable(batch_tensor).cuda()
cnn_feature = rpn_network.features(x)

Chúng ta nhận được cnn_features có kích thước là  (37 , 50)

In [ ]:
print cnn_feature.shape
print x.shape

In [ ]:
feature_height, feature_width = cnn_feature.shape[2], cnn_feature.shape[3]
im_height, im_width = x.shape[2] , x.shape[3]
batch_size = cnn_feature.shape[0]

### Nhận xét

Sau khi feed-forward một lần đối với ảnh gốc, thu được convolutional features của ảnh đó. Ví dụ với một hình ảnh có kích thước $600 * 800 * 3$, ta sẽ thu được convolutional features với kích thước $37 * 50 * 512$. Kích thước của features bị giảm nhỏ khoảng 16 lần $\frac{600}{37}$.

## AnchorTargerLayer

Sau khi thu được Convolutional Feature, chúng ta sẽ cùng tìm hiểu về AnchorTargetLayer.

### Vai trò của AnchorTargetlayer
    Gán cho anchor tương ứng với ground-truth boxes. 
    Tạo ra nhãn tương ứng cho từng anchor (bằng cách tính toán IOU)
    Tính giá trị mục tiêu của bounding-box regression

Tính toán đầu ra của RPN

- rpn_labels : (HxWxA, 1), for each anchor, 0 denotes bg, 1 fg, -1 dontcare
- rpn_bbox_targets: (HxWxA, 4), distances of the anchors to the gt_boxes(may contains some transform) that are the regression objectives
- rpn_bbox_inside_weights: (HxWxA, 4) weights of each boxes, mainly accepts hyper param in cfg
- rpn_bbox_outside_weights: (HxWxA, 4) used to balance the fg/bg, beacuse the numbers of bgs and fgs mays significiantly different

In [ ]:
from faster_rcnn.rpn_msr.anchor_target_layer_2 import AnchorTargerLayer
anchor_target_layer = AnchorTargerLayer(feat_stride=[16,], anchor_scales=[8, 16 ,32])

#### Tạo các anchor tương ứng với vị trí ảnh tại vị trí [0. , 0. , 0., 0.]

In [ ]:
features, rpn_bbox_pred, rpn_cls_score =  rpn_network._computer_forward(x)

In [ ]:
im_info = torch.Tensor([[im_height, im_width]])
batch_boxes_index = torch.Tensor(batch_boxes_index)
batch_boxes = torch.Tensor(batch_boxes).numpy()




In [ ]:
all_anchors = anchor_target_layer._create_anchors(feature_height, feature_width)
total_anchors = all_anchors.shape[0]

# only keep anchors inside the image
inside_anchors, inside_anchor_indexes = anchor_target_layer._filter_outside_anchors(
    all_anchors, im_height, im_width)
print all_anchors.shape

In [ ]:
target = anchor_target_layer.calculate_target(inside_anchors, batch_size, inside_anchor_indexes, batch_boxes, batch_boxes_index.numpy())


In [ ]:
result = anchor_target_layer((rpn_cls_score, torch.Tensor(batch_boxes), im_info, batch_boxes_index))
print(result[0][0].shape)

In [ ]:
print target[0].shape
print target[1].shape
print result[0][1].numpy() == 1
print all_anchors.shape

In [ ]:
i = 2
imshow(batch_tensor[i], blobs[i]['boxes'] ,inside_anchors[target[0][i] == 1] )
# imshow(batch_tensor[i], blobs[i]['boxes'] ,all_anchors[result[0][i].numpy() == 0] )